In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
import seaborn as sns
import time


from kan_convolutional.KANLinear import KANLinear
from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional import convolution 

In [2]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class Malimg(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.class_names = []

        for root_dir in root_dirs:
            for label, subfolder in enumerate(os.listdir(root_dir)):
                subfolder_path = os.path.join(root_dir, subfolder)
                if os.path.isdir(subfolder_path):
                    if subfolder not in self.class_names:
                        self.class_names.append(subfolder)
                    label = self.class_names.index(subfolder)
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.jpg'):
                            self.image_files.append(os.path.join(subfolder_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def get_class_names(self):
        return self.class_names

root_dirs = [
    # "C:\\Users\\Anurag Dutta\\Desktop\\Essentials\\Research\\malware\\gan_generated",
    "C:\\Users\\Anurag Dutta\\Desktop\\Essentials\\Research\\virus_mnist\\mnist"
]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = Malimg(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SuperConvolutionalKAN(nn.Module):
    def __init__(self):
        super(SuperConvolutionalKAN, self).__init__()

        self.conv1 = nn.Conv2d(1, 5, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(5, 25, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))

        self.flatten = nn.Flatten()

        self.kan1 = KANLinear(
            in_features=6400,
            out_features=256,
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

        self.kan2 = KANLinear(
            in_features=256,
            out_features=10,
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.maxpool(x)
        
        x = self.flatten(x)

        x = self.kan1(x)

        x = self.kan2(x)
        
        x = F.log_softmax(x, dim=1)

        return x
    
model = SuperConvolutionalKAN()


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(10): 
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_time = time.time() - epoch_start_time
    print(f'Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds')

total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
torch.save(model.state_dict(), 'super_kan_c_malware_classifier_64x64.pth')

Epoch [1/10], Loss: 0.3201, Time elapsed: 737.47 seconds
Epoch [2/10], Loss: 0.1690, Time elapsed: 1171.75 seconds
Epoch [3/10], Loss: 0.0972, Time elapsed: 739.26 seconds
Epoch [4/10], Loss: 0.0458, Time elapsed: 1045.46 seconds
Epoch [5/10], Loss: 0.0226, Time elapsed: 586.45 seconds
Epoch [6/10], Loss: 0.0165, Time elapsed: 582.68 seconds
Epoch [7/10], Loss: 0.0148, Time elapsed: 584.27 seconds
Epoch [8/10], Loss: 0.0342, Time elapsed: 571.56 seconds
Epoch [9/10], Loss: 0.0147, Time elapsed: 595.30 seconds
Epoch [10/10], Loss: 0.0164, Time elapsed: 600.64 seconds
Training completed in: 7214.86 seconds
Accuracy: 0.9185, Precision: 0.8914, Recall: 0.8748, F1 Score: 0.8748
